### **Secret이란?**




* 시크릿은 암호, 토큰 또는 키와 같은 소량의 중요한 데이터를 포함하는 오브젝트이다. 
* 이를 사용하지 않으면 중요한 정보가 [파드](https://kubernetes.io/ko/docs/concepts/workloads/pods/) 명세나 [컨테이너 이미지](https://kubernetes.io/ko/docs/reference/glossary/?all=true#term-image)에 포함될 수 있다. 
* 시크릿을 사용한다는 것은 사용자의 기밀 데이터를 애플리케이션 코드에 넣을 필요가 없음을 뜻한다.

* 시크릿은 시크릿을 사용하는 파드와 독립적으로 생성될 수 있기 때문에, 파드를 생성하고, 확인하고, 수정하는 워크플로우 동안 시크릿(그리고 데이터)이 노출되는 것에 대한 위험을 경감시킬 수 있다.
* 쿠버네티스 및 클러스터에서 실행되는 애플리케이션은 비밀 데이터를 비휘발성 저장소에 쓰는 것을 피하는 것과 같이, 시크릿에 대해 추가 예방 조치를 취할 수도 있다.

* 시크릿은 [컨피그맵](https://kubernetes.io/ko/docs/concepts/configuration/configmap/)과 유사하지만 특별히 기밀 데이터를 보관하기 위한 것이다.

### 시크릿 생성 방법

- `-from-literal`
- 시크릿 생성
    - `kubectl create secret generic my-password --from-literal password=p@ssw0rd`
        - generic <= Opaque 타입의 시크릿 생성을 명시
    
    
- 시크릿 확인
    - `kubectl get secrets`
    
- 시크릿 수정
    - `kubectl edit secret <secret name>`

**파일을 이용하여 패스워드 생성**

- 패스워드가 저장된 파일 2개 생성
    - `echo mypassword > pw1 && echo yourpassword > pw2`
    - `ls pw*`
- 시크릿 생성
    - `kubectl create secret generic our-password --from-file pw1 --from-file pw2`
    - `kubectl get secrets`
    
- 시크릿 내용 확인
    - `kubectl decribe secret our-password`

- `kubectl get secrets my-password -o yaml`
    
- BASE64 인코딩 및 디코딩
    - `echo p@ssw0rd | base64`
    
    
    - `echo cEBzc3cwcmQ= | base64 -d`
    

**Secret pod에 적용하기 - key에 파일명으로 사용하기**

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: alpine-secret
spec:
  containers:
    - name: alpine-secret
      image: alpine
      command: ["sleep"]
      args: ["100000"]
      env:
        - name: hello
          valueFrom:
            secretKeyRef:
              name: db-user-pass
              key: password.txt
```

**Secret을 volume에 추가하기**

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: alpine-secret-volume
spec:
  containers:
    - name: alpine-secret-volume
      image: alpine
      command: ["sleep"]
      args: ["100000"]
      volumeMounts:
        - name: foo
          mountPath: /etc/config
  volumes:
    - name: foo
      secret:
        secretName: my-password
        items:
          - key: password
            path: my-group/my-username
```


**Secret env에 추가해서 사용하기**

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: alpine-secret-env
spec:
  containers:
    - name: alpine-secret-env
      image: alpine
      command: ["sleep"]
      args: ["100000"]
      env:
        - name: password
          valueFrom:
            secretKeyRef:
              name: my-password
              key: password
```
